In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

## Full sentence data

In [2]:
file = '../input/bt5153-train-test-bert-full-sentence/train_data_bert_fullsent.csv'
testfile = '../input/bt5153-train-test-bert-full-sentence/test_data_bert_fullsent.csv'

df = pd.read_csv(file, dtype={'label':str})
df_test = pd.read_csv(testfile, dtype={'label':str})

In [3]:
# Merge DataFrames
df_all = pd.concat([df, df_test])
print(df.shape, df_test.shape, df_all.shape)
df_all.head(3)

(1065, 13) (458, 13) (1523, 13)


,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label
0,9161,"Delivery took more than a week, short expiry d...",2,1,1,0,"[-0.06329992, -0.32380387, 0.35608995, -0.1268...","[0.93797976, 0.25249124, 1.1054325, 0.49609792...","[-0.19600664, -0.4481723, 0.20494524, -0.39789...","[-0.046782605, -0.27473888, 0.2780265, 0.03694...","[0.4727717, 0.23103717, 0.7197178, 0.6083572, ...","[-0.022450736, -0.34166092, 0.13424423, -0.170...",110
1,8198,"Quality is so-so, loops are too huge resulting...",3,0,1,0,"[0.19020107, -0.3453789, 0.3494842, 0.14938574...","[1.1594802, 0.60506886, 1.084206, 0.8051733, 0...","[0.04546422, -0.29597518, -0.35977763, -0.4039...","[0.1399654, -0.11672436, 0.23911656, 0.1187961...","[0.8445691, 0.4961368, 0.7105234, 0.6109599, 0...","[0.12642525, -0.1857245, 0.08149249, -0.019622...",010
2,5314,Received within 3days. Well packed in a carton...,4,0,1,0,"[-0.028732965, -0.14712504, 0.36105582, 0.0254...","[1.3182856, 0.7953221, 1.3915914, 0.87499446, ...","[-0.38282838, -0.22101544, 0.6115111, -0.33658...","[0.07374873, -0.01620108, 0.20269494, 0.149083...","[0.76418287, 0.48008785, 0.97139245, 0.5615782...","[0.026645288, -0.27306008, 0.20750262, 0.05986...",010


## Setences with potisitve parts removed

In [4]:
df_neg = pd.read_csv('../input/5153-satge1-pos-neg-sent-clouds/1to4_stars_neg.csv') 
print(df_neg.shape)
df_neg.head()

(2072, 5)


,sentence,label,review_id,new_neg_comments,pred_label
0,"['ordered on 6th.', ""rec'd parcel on 10 feb at...",[0 0 0 0 0 0 0 0 1 0 1],2,"['ordered on 6th.', ""rec'd parcel on 10 feb at...",1
1,"['fast delivery!', 'first time buying.', 'it’s...",[1 1 1 0 0 0],3,"['the material is not too thick.', 'which i pr...",1
2,"['fast delivery.', 'box dented.', 'quality is ...",[1 0 0 0],12,"['box dented.', 'quality is thin.', 'i tried t...",1
3,"['second purchase the mask.', '.', 'disappoint...",[1 1 0 1 1 1],13,['disappointed not same as first i order.'],1
4,"['received within a few days.', '.', 'it is no...",[1 1 0 0 1 0],14,"['it is not 3 ply as described.', 'these are n...",1


In [5]:
df_neg_miss = df_neg[df_neg['new_neg_comments']=='[]']
df_neg_miss = df_neg_miss[df_neg_miss['review_id'].isin(df_all['review_id'])]
print(df_neg_miss.shape)
df_neg_miss.head(3)

(54, 5)


,sentence,label,review_id,new_neg_comments,pred_label
227,['would be better if packed individual'],[1],2002,[],0
231,"['it came a few week after ordering.', 'i was...",[1 1 1 1 1],2018,[],0
285,['seller not responsive on inquiries at all.'],[1],2452,[],0


In [6]:
df_neg_manual = df_all[df_all['review_id'].isin(df_neg_miss['review_id'])]
print(df_neg_manual.shape)
df_neg_manual.head(3)

(54, 13)


,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label
49,13077,Seconde time bought two box gave one free gift...,3,0,0,1,"[-0.08802131, -0.118899636, 0.22427808, 0.2316...","[0.4701014, 0.48993778, 0.56960297, 0.645804, ...","[-0.35550505, -0.009409012, -0.034824435, 0.20...","[0.043636676, -0.16437012, 0.16004011, 0.37967...","[0.54644644, 0.088169195, 0.62893945, 0.750418...","[-0.114293285, -0.21203962, 0.07086941, 0.1740...",001
75,23112,mask wasnt packed properly in the plastics :((...,4,0,1,1,"[-0.04009231, -0.28263184, 0.5977126, -0.15320...","[0.6928894, 0.36896294, 1.1612753, 0.39623103,...","[-0.025680419, -0.17138435, 0.47164458, 0.0370...","[0.048737098, -0.08047628, 0.39438245, 0.05233...","[0.665943, 0.2773788, 0.7819212, 0.38306567, 0...","[0.10069744, -0.17062676, 0.1513275, -0.099752...",011
86,32030,Comfy but very big,4,0,1,0,"[-0.3583474, -0.20432867, 0.19393235, 0.150656...","[0.54891074, 0.27769735, 0.5630641, 0.56893367...","[-0.76599616, 0.27769735, -0.025197128, -0.074...","[-0.06203093, -0.04217106, 0.210948, 0.2779134...","[0.7873698, 0.102550395, 0.47808596, 0.3849038...","[-0.20848864, -0.154832, 0.13418908, -0.069334...",010


In [7]:
print(len(list(df_neg_manual.review_content)))
list(df_neg_manual.review_content)

54


['Seconde time bought two box gave one free gift only.',
 'mask wasnt packed properly in the plastics :(( but mask packed in 10 which is nice and the colours are nice \nmask feels thin but hope it protects us',
 'Comfy but very big',
 'Received in good condition\nQuality so-so\nAbit rough',
 'Hope it can be a ziplock style bag ... overall quality is fine ...',
 'A bit tight overall okay',
 'Order kid hello kitty. Received kid plain pink mask. Anyway its ok.',
 'Delivery took more than a week.',
 'Material ok. delivery slightly longer.',
 'Not packed individually, quality feel ok.',
 'Well received, if can lower price abit will be perfect 👏🏻',
 'Sizing runs small and pretty thin. Value for money though.',
 'Bought 2 boxes at $16.90 each during Flash Sale. But now prices dropped to $13.00 per box.',
 'Very fast delivery but very strong smell',
 "19/01/21 Order , received on 26/01/31 wait around 7 day .1st buy , haven't try, hope is good.",
 'Ordered on 16 dec arrived on 18 dec. masks not

In [8]:
#manual create negative comments for the missing sentences
miss_neg_sentences = [
    ['two box gave one free gift only.'],
    ['mask wasnt packed properly in the plastics.', 'mask feels thin.'],
    ['very big.'],
    ['quality so-so.', 'a bit rough.'],
    ['hope it can be a ziplock style bag.'],
    ['a bit tight.'],
    ['order kid hello kitty.', 'received kid plain pink mask.'],
    ['delivery took more than a week.'],
    ['delivery slightly longer.'],
    ['not packed individually.'],
    ['if can lower price a bit.'],
    ['sizing runs small and pretty thin.'],
    ['but now prices dropped.'],
    ['very strong smell.'],
    ['wait around 7 day.'],
    ['masks not packed in packs.'],
    ['price is too high.'],
    ['seller stopped replying.', 'not good customer service.'],
    ['not easy to open the package.'],
    ['design not good.'],
    ['doesn’t fit well.'],
    ['received fast from the day i placed order.', 'item not properly sealed.'],
    ['workmanship not very good.'],
    ['feels like a satin mask.'],
    ['smell strong.', 'late delivery.'],
    ['proper packaging will be ideal.'],
    ['no box.'],
    ['i orders kids and received adult size.'],
    ['ordered 50pcs but only received 49.'],
    ['too huge.', 'such a pity.'],
    ['delivery a little long.'],
    ['dented box.'],
    ['prints not very clear.'],
    ['unfortunately the masks are not packed separately.'],
    ['the box looks like been crush.'],
    ['big.'],
    ['without a box.'],
    ['too thick.'],
    ['box dented.'],
    ['so so quality.'],
    ['Free collagen change from 2pcs to 1pc.'],
    ['a lot of dents and tears on the box.'],
    ['delivery late.'],
    ['delivery took a while.'],
    ['delivery quite long.', 'not packed in box.'],
    ['box not in good condition.'],
    ['breaks easily.'],
    ['came a few week after ordering.'],
    ['seller not responsive on inquiries at all.'],
    ['delivery not fast.'],
    ['would be better if packed individual.'],
    ['take one week for the delivery. '],
    ['masks seem pretty thin.', '3 ply only.'],
    ['really thick.']]

In [9]:
df_neg_manual['new_neg_comments'] = miss_neg_sentences
df_neg_manual.head(3)

,review_id,review_content,review_stars,delivery,product,service,bert_avg,bert_max,bert_layer0,distil_bert_avg,distil_bert_max,distil_bert_layer0,label,new_neg_comments
49,13077,Seconde time bought two box gave one free gift...,3,0,0,1,"[-0.08802131, -0.118899636, 0.22427808, 0.2316...","[0.4701014, 0.48993778, 0.56960297, 0.645804, ...","[-0.35550505, -0.009409012, -0.034824435, 0.20...","[0.043636676, -0.16437012, 0.16004011, 0.37967...","[0.54644644, 0.088169195, 0.62893945, 0.750418...","[-0.114293285, -0.21203962, 0.07086941, 0.1740...",001,[two box gave one free gift only.]
75,23112,mask wasnt packed properly in the plastics :((...,4,0,1,1,"[-0.04009231, -0.28263184, 0.5977126, -0.15320...","[0.6928894, 0.36896294, 1.1612753, 0.39623103,...","[-0.025680419, -0.17138435, 0.47164458, 0.0370...","[0.048737098, -0.08047628, 0.39438245, 0.05233...","[0.665943, 0.2773788, 0.7819212, 0.38306567, 0...","[0.10069744, -0.17062676, 0.1513275, -0.099752...",011,"[mask wasnt packed properly in the plastics., ..."
86,32030,Comfy but very big,4,0,1,0,"[-0.3583474, -0.20432867, 0.19393235, 0.150656...","[0.54891074, 0.27769735, 0.5630641, 0.56893367...","[-0.76599616, 0.27769735, -0.025197128, -0.074...","[-0.06203093, -0.04217106, 0.210948, 0.2779134...","[0.7873698, 0.102550395, 0.47808596, 0.3849038...","[-0.20848864, -0.154832, 0.13418908, -0.069334...",010,[very big.]


### Merge to get new_neg_comments for all sentences

In [10]:
use_cols = ['review_id', 'new_neg_comments']

df_remain = df_neg[df_neg['new_neg_comments']!='[]'][use_cols]
df_remain = df_remain[df_remain['review_id'].isin(df_all['review_id'])]

df_neg_merge = pd.concat([df_remain, df_neg_manual[use_cols]])
print(df_remain.shape, df_neg_manual.shape, df_neg_merge.shape)
df_neg_merge.head(3)

(1469, 2) (54, 14) (1523, 2)


,review_id,new_neg_comments
0,2,"['ordered on 6th.', ""rec'd parcel on 10 feb at..."
2,12,"['box dented.', 'quality is thin.', 'i tried t..."
3,13,['disappointed not same as first i order.']


In [11]:
df_all.dtypes

review_id              int64
review_content        object
review_stars           int64
delivery               int64
product                int64
service                int64
bert_avg              object
bert_max              object
bert_layer0           object
distil_bert_avg       object
distil_bert_max       object
distil_bert_layer0    object
label                 object
dtype: object

In [12]:
df_all = df_neg_merge.rename(columns = {'new_neg_comments': 'review_content'})
df_all['review_content'] = df_all['review_content'].astype('str')
df_all['neg_review_content'] = df_all['review_content'].apply(lambda x : ' '.join(eval(x)))
df_all.head(3)

,review_id,review_content,neg_review_content
0,2,"['ordered on 6th.', ""rec'd parcel on 10 feb at...",ordered on 6th. rec'd parcel on 10 feb at 11.3...
2,12,"['box dented.', 'quality is thin.', 'i tried t...",box dented. quality is thin. i tried to wear n...
3,13,['disappointed not same as first i order.'],disappointed not same as first i order.


In [13]:
df_all.to_csv('negative_review_content.csv', index=False)

# (1) BERT Feature Extraction - BertTokenizer

In [14]:
!pip install transformers

In [15]:
import torch
from transformers import BertTokenizer, BertModel

In [16]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model tokenizer (vocabulary)
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
#Step 1: Tokenization
tokenized = df_all['review_content'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
#Step 2: Padding
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print(padded.shape)
print(padded[0])
print(tokenizer.convert_ids_to_tokens(0))

(1523, 106)
[  101  1031  1005  3641  2006  5351  1012  1005  1010  1000 28667  1005
  1040 20463  2006  2184 13114  2012  2340  1012  2382  9737  1012  1000
  1010  1005 28516  1018  8378  1012  1005  1010  1005  1015  3482  2234
  4064  1012  1005  1010  1005  2025  2469  2339  2064  5308  6229  2066
  2008  1029  1005  1010  1005  5653  2041  4064  3482  1012  1005  1010
  1005  5796  2290 14939  3294  2053  3433  2012  2035  1012  1005  1010
  1005  6406  2005 25416  8630  1012  1005  1010  1005  4497  4402  8285
 14300 25416  8630  2144  2053  3433 10424 14939  1012  1005  1033   102
     0     0     0     0     0     0     0     0     0     0]
[PAD]


In [19]:
#Step 3: Masking
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1523, 106)

In [20]:
%%time
#Step 4: Use the pre-trained BERT Model
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    ## the output here is a tuple of (last_hidden_state, pooler_output)
    model_outputs = model(input_ids, attention_mask=attention_mask)

CPU times: user 4min 32s, sys: 1min 22s, total: 5min 54s
Wall time: 5min 59s


In [21]:
# get the last_hidden state
last_hidden_state = model_outputs[0].numpy()
last_hidden_state.shape

(1523, 106, 768)

### Save results

In [22]:
def save_to_dic(keys, values):
    dic = {}
    for i, key in enumerate(keys):
        dic[key] = list(values[i])
    return dic

In [23]:
def save_dic_to_txt(save_name, dic):
    f = open(save_name, 'w')
    f.write(str(dic))
    f.close()

In [24]:
bert_dic_2D = save_to_dic(df_all['review_id'], last_hidden_state)
#save_dic_to_txt('negative_bert_dic_2D.txt', bert_dic_2D)
f_save = open('negative_bert_dic_2D.pkl', 'wb')
pickle.dump(bert_dic_2D, f_save)
f_save.close()

In [25]:
#Get the embeddings for position 0
bert_layer0 = last_hidden_state[:,0,:]
print(bert_layer0.shape)

bert_layer0_dic = save_to_dic(df_all['review_id'], bert_layer0)
save_dic_to_txt('negative_bert_layer0_dic.txt', bert_layer0_dic)

(1523, 768)


In [26]:
#Get the embeddings for avg
bert_avg = np.mean(last_hidden_state,axis=1)
print(bert_avg.shape)

bert_avg_dic = save_to_dic(df_all['review_id'], bert_avg)
save_dic_to_txt('negative_bert_avg_dic.txt', bert_avg_dic)

(1523, 768)


In [27]:
#Get the embeddings for max
bert_max = np.max(last_hidden_state,axis=1)
print(bert_max.shape)

bert_max_dic = save_to_dic(df_all['review_id'], bert_max)
save_dic_to_txt('negative_bert_max_dic.txt', bert_max_dic)

(1523, 768)


# (2) BERT Feature Extraction - DistilBert

In [28]:
from transformers import DistilBertTokenizer, DistilBertModel
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load pre-trained model (weights)
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [29]:
tokenized = df_all['review_content'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1523, 106)

In [30]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [31]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1523, 106)

In [32]:
%%time
feature_list_layer0 = []
feature_list_avg = []
feature_list_max = []
feature_list_2d = []
with torch.no_grad():
    for batch_idx in range(0,padded.shape[0]):
        #BERT check 10 sample each time.
        input_ids = torch.tensor(padded[batch_idx:batch_idx+1])  
        used_attention_mask = torch.tensor(attention_mask[batch_idx:batch_idx+1])
        last_hidden_states = model(input_ids, attention_mask=used_attention_mask)
        #Get the embeddings for the [CLS] tag (position is 0)
        features_layer0 = last_hidden_states[0][:,0,:].numpy()
        features_avg = np.mean(last_hidden_states[0].numpy(),axis=1)
        features_max = np.max(last_hidden_states[0].numpy(),axis=1)
        features_2d = last_hidden_states[0].numpy()
        feature_list_layer0.append(features_layer0)
        feature_list_avg.append(features_avg)
        feature_list_max.append(features_max)
        feature_list_2d.append(features_2d)

CPU times: user 3min 22s, sys: 1.22 s, total: 3min 23s
Wall time: 3min 25s


In [33]:
# preprare features
features_layer0 = np.vstack(feature_list_layer0)
print(features_layer0.shape)

features_avg = np.vstack(feature_list_avg)
print(features_avg.shape)

features_max = np.vstack(feature_list_max)
print(features_max.shape)

(1523, 768)
(1523, 768)
(1523, 768)


In [34]:
# save to files
distil_bert_layer0_dic = save_to_dic(df_all['review_id'], features_layer0)
save_dic_to_txt('negative_distil_bert_layer0_dic.txt', distil_bert_layer0_dic)

distil_bert_avg_dic = save_to_dic(df_all['review_id'], features_avg)
save_dic_to_txt('negative_distil_bert_avg_dic.txt', distil_bert_avg_dic)

distil_bert_max_dic = save_to_dic(df_all['review_id'], features_max)
save_dic_to_txt('negative_distil_bert_max_dic.txt', distil_bert_max_dic)

distil_bert_dic_2D = save_to_dic(df_all['review_id'], feature_list_2d)
#save_dic_to_txt('negative_distil_bert_dic_2D.txt', distil_bert_dic_2D)
f_save = open('distil_bert_dic_2D.pkl', 'wb')
pickle.dump(distil_bert_dic_2D, f_save)
f_save.close()